Implement a well formed substrings table.

In [1]:
from tabulate import tabulate
import random
import string


def gen_symbol(length):
    return f"NewT_{''.join(random.choices(string.ascii_uppercase + string.digits, k=length))}"


def split_rule(lhs, rhs):
    if len(rhs) > 2:
        rules = []
        new_sym = gen_symbol(len(rhs) - 1)
        rules.append((lhs, [rhs[0], new_sym]))
        for i in range(1, len(rhs) - 2):
            next_sym = gen_symbol(len(rhs) - i - 1)
            rules.append((new_sym, [rhs[i], next_sym]))
            new_sym = next_sym
        rules.append((new_sym, rhs[-2:]))
        return rules
    else:
        return [(lhs, rhs)]


def process_rules(rules):
    processed = []
    for lhs, rhs in rules:
        processed.extend(split_rule(lhs, rhs))
    return processed


def init_table(size):
    return [[[] for _ in range(size + 1)] for _ in range(size + 1)]


def fill_terminals(table, words, rules):
    for i, word in enumerate(words):
        for lhs, rhs in rules:
            if len(rhs) == 1 and rhs[0] == word:
                table[i][i + 1].append((lhs, rhs))


def apply_rules(table, rules, n):
    for length in range(2, n + 1):
        for start in range(n - length + 1):
            end = start + length
            for mid in range(start + 1, end):
                for left1, _ in table[start][mid]:
                    for left2, _ in table[mid][end]:
                        for lhs, rhs in rules:
                            if len(rhs) == 2 and rhs == [left1, left2]:
                                if (lhs, rhs) not in table[start][end]:
                                    table[start][end].append((lhs, rhs))


def parse_sentence(sentence, rules):
    words = sentence.split()
    n = len(words)
    table = init_table(n)

    processed_rules = process_rules(rules)
    print("Rules:")
    for lhs, rhs in processed_rules:
        print(f"{lhs} -> {' '.join(rhs)}")

    fill_terminals(table, words, processed_rules)
    apply_rules(table, processed_rules, n)

    return table


def show_table(table):
    formatted = [
        ["" if cell == [] else cell for cell in row]
        for row in table
    ]
    print(tabulate(formatted, tablefmt="grid"))


rules = [
    ('S', ['NP', 'VP']),
    ('NP', ['Det', 'N']),
    ('NP', ['Det', 'Adj', 'N']),
    ('VP', ['V', 'Adv']),
    ('VP', ['V', 'PP']),
    ('PP', ['P', 'NP']),
    ('Det', ['a']),
    ('N', ['cat']),
    ('N', ['fence']),
    ('Adj', ['small']),
    ('V', ['jumped']),
    ('Adv', ['quickly']),
    ('P', ['over'])
]

sentence = "a small cat quickly jumped over a fence"

table = parse_sentence(sentence, rules)
show_table(table)


Rules:
S -> NP VP
NP -> Det N
NP -> Det NewT_FV
NewT_FV -> Adj N
VP -> V Adv
VP -> V PP
PP -> P NP
Det -> a
N -> cat
N -> fence
Adj -> small
V -> jumped
Adv -> quickly
P -> over
+--+------------------+----------------------+------------------------------+------------------------+---------------------+-------------------+------------------+------------------------+
|  | [('Det', ['a'])] |                      | [('NP', ['Det', 'NewT_FV'])] |                        |                     |                   |                  |                        |
+--+------------------+----------------------+------------------------------+------------------------+---------------------+-------------------+------------------+------------------------+
|  |                  | [('Adj', ['small'])] | [('NewT_FV', ['Adj', 'N'])]  |                        |                     |                   |                  |                        |
+--+------------------+----------------------+--------------------